In [ ]:
def tf_preprocess(image):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, (32, 32))
  image = image[None, ...]
  return image

# 확률 벡터에서 레이블을 추출해주는 헬퍼 메서드
def get_tf_label(labels):
    label = tf.cast(labels, tf.int32)
    label = tf.reshape(label,[1,12])
    return label

def create_adversarial_pattern(input_image, input_label):
  with tf.GradientTape() as tape:
    tape.watch(input_image)
    input_img = tf.reshape(input_image,[1,32,32,3])
    prediction = model(input_img)
    loss = loss_object(input_label, prediction)

  # 입력 이미지에 대한 손실 함수의 기울기를 구합니다.
  gradient = tape.gradient(loss, input_image)
  # 왜곡을 생성하기 위해 그래디언트의 부호를 구합니다.
  signed_grad = tf.sign(gradient)
  return signed_grad

def fgsm_attack(model,test_x,test_y,eps):
    
    prog = 0 # 진행상황 확인용 변수

    correct = 0
    adv_examples = []
    save_adv_examples = [] # 공격받은 이미지들이 저장될 리스트
    save_original_output = [] # 공격받은 이미지들의 정답 라벨 값이 저장될 리스트
    
    for i in range(len(test_x)):
        # 1장의 이미지와 그 label
        data = test_x[i]
        target_onehot = test_y[i] # one-hot 형태
        target_label = int(np.argmax(target_onehot)) # label 형태

        # model이 정상 데이터를 분류한 결과 (각각 one-hot 형태, int label 형태)
        result_onehot = model.predict(data.reshape(1,32,32,3)) # one-hot 형태
        result_label = int(np.argmax(result_onehot))

        # 모델이 정상 데이터인데도 잘못 분류했다면 사용하지 않는다 (아래 코드 실행하지 않고 다음 이미지로 넘어감)
        if target_label != result_label:
            continue

        # 이미지 전처리
        img =  tf_preprocess(data) # 텐서플로 전처리
        label = get_tf_label(target_onehot) # 확률벡터에서 레이블 추출
        
        # FGSM 공격 수행
        perturbations = create_adversarial_pattern(img, label)
        adv_x = img + eps * perturbations
        adv_x = tf.clip_by_value(adv_x, 0, 1) # 공격받은 이미지

        # 공격 이미지를 분류기에 넣은 결과; 잘못 분류되어야 할 것 
        atkresult_onehot = model.predict(adv_x) # one-hot 형태
        atkresult_label = int(np.argmax(atkresult_onehot)) # label 형태

        # 만약 공격 받아도 제대로 분류된다면 correct로 count
        if atkresult_label == target_label:
            correct += 1
        
        # 공격 적용된 이미지, 그 공격 받은 이미지의 원래 정답 label을 각각 리스트에 저장합니다
        save_adv_examples.append(tf.reshape(adv_x,[32,32,3]))
        save_original_output.append(target_label)

        prog += 1

        if prog%500 == 0:
          print("prog :", prog)

    # 해당 엡실론에서의 최종 정확도를 계산합니다
    final_acc = correct/float(len(test_x))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(eps, correct, len(test_x), final_acc))

    # 정확도와 적대적 예제를 리턴합니다
    return final_acc, adv_examples, save_adv_examples, save_original_output

def pgd_attack(model,test_x,test_y,eps,step_size=2,num_steps=7): 
    """
    FGSM 코드와 차이점
    - step_size, num_steps 파라미터 추가됨
    - unifrom distribution 코드 추가
    - FGSM 공격 수행 -> PGD 공격 수행
    ** 모든 return 형식은 동일함
    
    default 값
    - step_size = 2 (alpha 값)
    - num_steps = 7 (iterations 값)

    """

    prog = 0 # 진행상황 확인용 변수

    correct = 0
    adv_examples = []
    save_adv_examples = [] # 공격받은 이미지들이 저장될 리스트
    save_original_output = [] # 공격받은 이미지들의 정답 라벨 값이 저장될 리스트
    
    for i in range(len(test_x)):
        # 1장의 이미지와 그 label
        data = test_x[i]
        target_onehot = test_y[i] # one-hot 형태
        target_label = int(np.argmax(target_onehot)) # label 형태

        # model이 정상 데이터를 분류한 결과 (각각 one-hot 형태, int label 형태)
        result_onehot = model.predict(data.reshape(1,32,32,3)) # one-hot 형태
        result_label = int(np.argmax(result_onehot))

        # 모델이 정상 데이터인데도 잘못 분류했다면 사용하지 않는다 (아래 코드 실행하지 않고 다음 이미지로 넘어감)
        if target_label != result_label:
            continue

        # PGD uniform distribution 코드
        data = data + np.random.uniform(-eps,eps,data.shape)
        data = np.clip(data,0,255)

        # 이미지 전처리
        img =  tf_preprocess(data) # 텐서플로 전처리 -> 0~1사이로 정규화 함.
        label = get_tf_label(target_onehot) # 확률벡터에서 레이블 추출
        
        # PGD 공격 수행
        adv_x = img # 공격받은 이미지 (for문으로 업데이트)
        for num_step in range(num_steps):
          perturbations = create_adversarial_pattern(adv_x,label) # signed_grad를 리턴한 값
          adv_x += step_size * perturbations
          adv_x = tf.clip_by_value(adv_x,img-eps,img+eps)
          adv_x = tf.clip_by_value(adv_x,0,1)

        # 공격 이미지를 분류기에 넣은 결과; 잘못 분류되어야 할 것 
        atkresult_onehot = model.predict(adv_x) # one-hot 형태
        atkresult_label = int(np.argmax(atkresult_onehot)) # label 형태

        # 만약 공격 받아도 제대로 분류된다면 correct로 count
        if atkresult_label == target_label:
            correct += 1
        
        # 공격 적용된 이미지, 그 공격 받은 이미지의 원래 정답 label을 각각 리스트에 저장합니다
        save_adv_examples.append(tf.reshape(adv_x,[32,32,3]))
        save_original_output.append(target_label) #공격 받은 이미지에 원래 타겟 값을 저장합니다 ! 

        prog += 1

        if prog%500 == 0:
          print("prog :", prog)

    # 해당 엡실론에서의 최종 정확도를 계산합니다
    final_acc = correct/float(len(test_x))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(eps, correct, len(test_x), final_acc))

    # 정확도와 적대적 예제를 리턴합니다
    return final_acc, adv_examples, save_adv_examples, save_original_output

def Fourier_transform(img):
  f = np.fft.fft2(img) # 이미지에 푸리에 변환 적용
  fshift = np.fft.fftshift(f) # 분석을 용이하게 하기 위해 주파수가 0인 부분을 중앙에 위치시킴. 중앙에 저주파가 모이게 됨.
  
  mag_spectrum = 20*np.log(np.abs(fshift)) # spectrum 구하는 수학식.
  if np.min(mag_spectrum) < 0:
    mag_spectrum -= np.min(mag_spectrum) # 0~???로 변환

  if np.min(mag_spectrum) < 0: # 확인용; 프린트 X
    print("### 나오면 안됨 ### 0보다 작은 값 안 없어짐", np.min(mag_spectrum))

  maxval = np.max(mag_spectrum)
  if maxval <= 0:
    print("### maxval 0 이하인 경우 있음 ### ")
    maxval += 1e-6

  mag_spectrum = mag_spectrum/maxval # 0~1 정규화

  if np.max(mag_spectrum) < 0 or np.min(mag_spectrum) < 0:
    print("### 0보다 작은 값 남아있음 ###")
  
  mag_spectrum = mag_spectrum * 255
  mag_spectrum = (mag_spectrum.astype(int))
  # print("min :", np.min(mag_spectrum), "max :", np.max(mag_spectrum))

  if np.max(mag_spectrum) > 255: # 확인용; 프린트 X
    print("### 나오면 안됨 ### 1보다 큰 값 있음")

  return mag_spectrum